<a href="https://colab.research.google.com/github/v2shanker/pcps_vax/blob/master/RBD_Nmer_Combs_Varun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions to Run**



1.   Change parameter and input values as necessary, in cell indicated below
2.   In Toolbar above, click Runtime-->Run All 
3.   Wait for all cells to execute in order for output



In [13]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     |████████████████████████████████| 2.3MB 3.5MB/s 


In [27]:
import os
import sys
from itertools import combinations 
from itertools import permutations
import pprint
from itertools import groupby
import pandas as pd

In [16]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

----------------------------------------------------------------------
# CHANGE PARAMETER VALUES HERE
|parameter|description|type|
|:-------|:----------:|:----------:|
|`nmers_of_interest` | list of allowed lengths of PCPS products|`list of int`|
|`tolerance` | hit-calling range permissible from original peak value|`float`|
|`flat_parent`| parental peptide sequence|`string`|
|`IP_Peaks_List` | list of candidate peaks under evaluation|`list of float`|
|`max_num_fragments` | upper bound for number of frgaments in PCPS product|`int`|

*n.b. adding more values to lists like `nmers_of_interest` and `IP_Peaks_List` will significantly increase runtime*


In [63]:
# possible n_mers
# ADD OR SUBTRACT HERE VALUES IF WANT MORE N-MERS
nmers_of_interest = [8,9,10]
#CHANGE SEQUENCE HERE OF PEPTIDE
flat_parent = 'FTGCVIAWNSNNLDSKVGGNYNYLYC'

#CHANGE TOLERANCE HERE!
tolerance = 1.5

#LIST ALL PEAKS HERE TO ANALYZE 
IP_Peaks_List = [1000,1008]

#CHANGE THE UPPER BOUND OF FRAGMENTS FROM WHICH A PCPS PRODUCT CAN BE COMPOSED OF
max_num_frags = 2

----------------------------------------------------

In [54]:
parent = []
for i in flat_parent:
    parent.append(i)

In [55]:
#create list of all combinations of the given n_mers of interest and SUBTRACTS duplicate combinations 
all_seq_mers = []
for size in nmers_of_interest:
    seq_n_mers = combinations(parent, size) 
    seq_n_mers = list(dict.fromkeys(seq_n_mers)) #deletes duplicates by dict. conversion
    all_seq_mers.extend(seq_n_mers) #create list that adds 8mers, then 9mers, then 10mers...

In [56]:
#computes the molecular weight of a given Amino Acids Sequence 
def compute_mol_weight(AA_comb):
    AA_DICT = {
        'A' : 71.09, 
        'R' : 156.19, 
        'D' : 114.11,
        'N' : 115.09,
        'C' : 103.15, 
        'E' : 129.12,
        'Q' : 128.14,
        'G' : 57.05,
        'H' : 137.14,
        'I' : 113.16,
        'L' : 113.16,
        'K' : 128.17,
        'M' : 131.19,
        'F' : 147.18,
        'P' : 97.12,
        'S' : 87.08,
        'T' : 101.11,
        'W' : 186.12,
        'Y' : 163.18,
        'V' : 99.14
    }
    
    mw = 0
    for AA in AA_comb:
        mw = mw + AA_DICT[AA] + 18.01528 #accounts for terminal hydration

    return mw

In [57]:
#Create a dictionary with key values as all possible n-mer sequences and value as the molecular weight
comb_dict = {}
for seq_comb in all_seq_mers:
    comb_dict[seq_comb] = compute_mol_weight(seq_comb)

In [58]:
# DO NOT CHANGE ANYTHING HERE!!!
def find_peak_seq_combs(peak,tolerance):
    '''This function takes in a peak value and a tolerance and returns the possible matches w/ or w/o modifications
    from the list of all possible recombinations of the sequence and desired peptide recombination 
    lengths entered above
    '''
    peak_sequence_hits = {}
    for seq_comb in comb_dict:
        if abs(peak - comb_dict[seq_comb]) <= tolerance:
            peak_sequence_hits[''.join(seq_comb)] = [comb_dict[seq_comb] , len(seq_comb)]
            
            
            
#         elif abs(peak - (comb_dict[seq_comb] - 1)) <= tolerance:
#             peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Loss Proton', len(seq_comb)]
#         elif abs(peak - (comb_dict[seq_comb] + 1)) <= tolerance:
#             peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Gain Proton', len(seq_comb)]
#         elif abs(peak - (comb_dict[seq_comb] + 42)) <= tolerance:
#             peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Acetylation', len(seq_comb)]
#         elif abs(peak - (comb_dict[seq_comb] + 18.01528)) <= tolerance:
#             peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Gain Water', len(seq_comb)]
#         elif abs(peak - (comb_dict[seq_comb] + 22.99)) <= tolerance:
#             peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Gain Sodium,', len(seq_comb)]
#         elif abs(peak - (comb_dict[seq_comb] + 22.99 + 18.01528)) <= tolerance:
#              peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Gain Sodium, Gain Water', len(seq_comb)]
#         elif abs(peak - (comb_dict[seq_comb] + 22.99 + 1)) <= tolerance:
#             peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Gain Sodium, Gain Proton', len(seq_comb)]
#         elif abs(peak - (comb_dict[seq_comb] + 22.99 - 1)) <= tolerance:
#             peak_sequence_hits[seq_comb] = [comb_dict[seq_comb] , 'Gain Sodium, Loss Proton', len(seq_comb)]
        
    print ("There are " + str(len((peak_sequence_hits))) + " possible recombination products")    
    #prints all AA combinations that are a hit for peak
    #pprint.pprint(peak_sequence_hits)
    
    
    #create list of all hit sequences
    all_hit_sequences =[]
    for hit_seq in list( peak_sequence_hits.keys()):
        all_hit_sequences.append(hit_seq)

    
    return all_hit_sequences

In [59]:
def alignment_hit_seq(all_hit_sequences):
    '''Takes all possible AA combos that match a peak and returns best alignment(s)'''
    
    #finds best alignment for every possible AA combination that is a hit for peak
    best_alignments = []
    for hit_seq in all_hit_sequences:
        #Identical characters are given 2 points, 1 point is deducted for each non-identical character.
        #0.5 points are deducted when opening a gap, and 0.1 points are deducted when extending it.
        best_alignments.extend(pairwise2.align.globalms(flat_parent,hit_seq, 2, -1, -.5, -.1))

#         for a in pairwise2.align.globalms(flat_parent,hit_seq, 2, -1, -.5, -.1):
#             print(format_alignment(*a))
#         print("next")
                                    

    return best_alignments


In [60]:
def alignment_to_frag(best_aligments):
    '''Takes in all possible aligned AA combs for a peak, and returns the unique set of fragments for the peak'''
    seq_frags = []
    for a in best_alignments:
        aligned_seq = a[1]
        fragments = [i for i in aligned_seq.split('-') if i != ""]
        seq_frags.append(fragments)
    
    
    #get rid of duplicate sets of fragments
    seq_frags.sort()
    dedup_seq_frags = list(seq_frags for seq_frags,_ in groupby(seq_frags))

    
    return dedup_seq_frags
        

In [61]:
def frag_to_PCPS_lib(dedup_seq_frags, max_num_frag):
    
    PCPS_library = []
    for frags in dedup_seq_frags:
        #if the set of fragments is less than threshold
        if len(frags) <= max_num_frag:
            #find all PCPS rearragements of this set of fragments
            frag_PCPS_seqs = list(permutations(frags,len(frags)))
            #join the rearraged fragments into a string and add to library
            for PCPS_seq in frag_PCPS_seqs:
                PCPS_library.append(''.join(list(PCPS_seq)))
    
    return PCPS_library


In [66]:
for peak in IP_Peaks_List:
    print('---------------Results for Peak '+ str(peak) + ' in --------------------------------------')
    all_hit_sequences = find_peak_seq_combs(peak,tolerance)
    
    best_alignments = alignment_hit_seq(all_hit_sequences)
    dedup_seq_frags = alignment_to_frag(best_alignments)
    
    PCPS_lib = frag_to_PCPS_lib(dedup_seq_frags, max_num_frags)  
    lib_len = len(PCPS_lib)
    print(f'There are {lib_len} hits composed of at most {max_num_frags} fragments')
    print(*PCPS_lib,sep='\n')
    
    print('\n');



---------------Results for Peak 1000 in --------------------------------------
There are 4608 possible recombination products
There are 30 hits composed of at most 2 fragments
AGGNYNYL
GGNYNYLA
AWLDSKVG
LDSKVGAW
CVILDSKV
LDSKVCVI
CVIAWKVG
KVGCVIAW
FTVGGNYN
VGGNYNFT
FTGCSKVGG
SKVGGFTGC
FTGCVIAY
YFTGCVIA
GCVIAWKV
KVGCVIAW
GCVIAWLD
LDGCVIAW
IAGGNYNY
GGNYNYIA
IANNLDSK
NNLDSKIA
IAWDSKVG
DSKVGIAW
IAWNSKVG
SKVGIAWN
IAWNSKVG
KVGIAWNS
SVGGNYNY
VGGNYNYS


---------------Results for Peak 1008 in --------------------------------------
There are 9387 possible recombination products
There are 26 hits composed of at most 2 fragments
AWVGGNYN
VGGNYNAW
AWNVGGNY
VGGNYAWN
CSNNLDSK
SNNLDSKC
DSKVGLYC
LYCDSKVG
FTGNLDSK
NLDSKFTG
LDSKVGYC
YCLDSKVG
SNNLDSVGG
VGGSNNLDS
SNNLDSKC
CSNNLDSK
TGCVDSKVG
DSKVGTGCV
TGCVWNSN
WNSNTGCV
TGCVINYL
NYLTGCVI
TGCVISKVG
SKVGTGCVI
TGCVIAWGG
GGTGCVIAW




In [52]:
for a in pairwise2.align.globalms("ANRGDFS", "ANREFS", 2, -1, -.5, -.1):
    print(format_alignment(*a))
    print(a)

ANRGD-FS
|||   ||
ANR--EFS
  Score=8.9

Alignment(seqA='ANRGD-FS', seqB='ANR--EFS', score=8.9, start=0, end=8)
